In [112]:
def parse_text(text: str) -> str:
    return text.replace("**", "{b}").replace("*", "{i}")


def close_tags(text: str) -> str:
    _open = False
    open_chars = set()
    output = ""
    for char in text:
        if char == "{":
            _open = True
        elif char == "}":
            _open = False
        elif _open:
            if char not in open_chars:
                open_chars.add(char)
                output += "{" + char + "}"
            else:
                open_chars.remove(char)
                output += "{/" + char + "}"
        else:
            output += char
    return output

skip_keywords = ["!", "."]

characters = {
    "DAISY-BOT": ("daisy", "left"),
    "BATTA S. DEVILA": ("batta", "right"),
    "MOON DOG CHEF": ("dog", "center"),
}

In [113]:
with open("Toxic Yuri Game Jam 2025_ Script.fountain", "r") as file:
    text = file.read()

In [114]:
output: str = ""
character = None
current_string = ""
characters_present = set()

for line in text.split("\n"):
    line = line.strip()
    if not line:
        if character or current_string:
            output += "\t"
            if character:
                output += character + " "
            if current_string:
                output += f"\"{close_tags(current_string.strip())}\""
            output += "\n"
        character = None
        current_string = ""
        continue
    if character:
        current_string += parse_text(line) + " "
    else:
        character_found = False
        for char_text, (name, pos) in characters.items():
            if line.startswith(char_text):
                character = name

                expression = ""
                if "(" in line:
                    expression = " " + (
                        line[line.find("(") + 1 : line.find(")")].replace("-", "").lower()
                    )

                if character not in characters_present:
                    output += f"\tshow {name}{expression} at {pos}\n"
                    characters_present.add(character)
                elif expression:
                    output += f"\tshow {name}{expression}\n"
                break
        if not character:
            if any(line.startswith(keyword) for keyword in skip_keywords):
                continue
            output += f"\t\"{line.strip()}\"\n"
            continue

print(output)

	show batta relaxed happy at right
	batta "Oh DAISY-BOT, my precious little iron pill, you would not {i}believe{/i} the dream I just had-"
	show daisy neutral at left
	daisy "My Lady, surely this might not be another one of your {i}fantasy dreams?{/i} Because I feel as your Chief of Staff, it is my duty to remind you that the floors in the West Hall corridor are {i}still{/i} not clean from the last one-"
	show batta relaxed flustered
	batta "No, no no! Not one of those, although..."
	show batta relaxed sultry
	batta "... I certainly wouldn't mind a repeat of last week's... experiment."
	show daisy flustered
	daisy "My Lady-"
	show batta relaxed sultry
	batta "Not when gobbling you up satisfies almost as much as a freshly- bitten throat, my darling little bug~ Like licking between two bright new copper pennies every time I go d-"
	show batta relaxed confused
	batta "-wait, that {i}was{/i} a week ago. What do you mean the West Hall still hasn't been cleaned yet?"
	show daisy neutral
	dai